In [1]:
!pip install pandas xlrd ipdb

In [2]:
import os
import pandas as pd
import ipdb
from datetime import datetime, time

In [3]:
from index import get_df_with_opening_hours

In [4]:
%ls data
fn = 'data/weekly-2020-02-10 0925.xlsx'
new_df = get_df_with_opening_hours(fn)
new_df

weekly-2020-02-10 0925.xlsx


,datetime,library,people_in,people_out,opening_type
0,2020-02-03 00:00:00,Chipping Barnet,0,0,closed
6,2020-02-03 00:00:00,Colindale,0,0,closed
9,2020-02-03 00:00:00,East Finchley,0,0,closed
12,2020-02-03 00:00:00,Edgware,0,0,closed
15,2020-02-03 00:00:00,Golders Green,0,0,closed
...,...,...,...,...,...
561,2020-02-09 23:00:00,East Finchley,0,0,closed
564,2020-02-09 23:00:00,Edgware,0,0,closed
567,2020-02-09 23:00:00,Golders Green,0,0,closed
570,2020-02-09 23:00:00,North Finchley,0,0,closed


In [5]:
curr = pd.read_csv('2020.csv')
curr

,datetime,library,people_in,people_out,opening_type
0,2020-01-01 00:00:00,Chipping Barnet,0,0,closed
1,2020-01-01 00:00:00,Colindale,0,0,closed
2,2020-01-01 00:00:00,Edgware,0,0,closed
3,2020-01-01 00:00:00,Finchley Church End,0,0,closed
4,2020-01-01 00:00:00,Golders Green,0,0,closed
...,...,...,...,...,...
10243,2020-02-23 23:00:00,Edgware,0,0,closed
10244,2020-02-23 23:00:00,Finchley Church End,0,0,closed
10245,2020-02-23 23:00:00,Golders Green,0,0,closed
10246,2020-02-23 23:00:00,North Finchley,0,0,closed


In [6]:
res = pd.concat([curr, new_df])
res

,datetime,library,people_in,people_out,opening_type
0,2020-01-01 00:00:00,Chipping Barnet,0,0,closed
1,2020-01-01 00:00:00,Colindale,0,0,closed
2,2020-01-01 00:00:00,Edgware,0,0,closed
3,2020-01-01 00:00:00,Finchley Church End,0,0,closed
4,2020-01-01 00:00:00,Golders Green,0,0,closed
...,...,...,...,...,...
561,2020-02-09 23:00:00,East Finchley,0,0,closed
564,2020-02-09 23:00:00,Edgware,0,0,closed
567,2020-02-09 23:00:00,Golders Green,0,0,closed
570,2020-02-09 23:00:00,North Finchley,0,0,closed


In [7]:
res.drop_duplicates(inplace=True)
res

,datetime,library,people_in,people_out,opening_type
0,2020-01-01 00:00:00,Chipping Barnet,0,0,closed
1,2020-01-01 00:00:00,Colindale,0,0,closed
2,2020-01-01 00:00:00,Edgware,0,0,closed
3,2020-01-01 00:00:00,Finchley Church End,0,0,closed
4,2020-01-01 00:00:00,Golders Green,0,0,closed
...,...,...,...,...,...
561,2020-02-09 23:00:00,East Finchley,0,0,closed
564,2020-02-09 23:00:00,Edgware,0,0,closed
567,2020-02-09 23:00:00,Golders Green,0,0,closed
570,2020-02-09 23:00:00,North Finchley,0,0,closed


In [8]:
res.to_csv('2020.csv', index=False)

In [9]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
